In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [2]:
path_1 = './max/'
path_2 = './max20160907'

this_path = path_2

In [3]:
def get_X_y_from(t_extract_data, t_data, t_estims_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [t_extract_data(t_data[clinic_ids.index(i['id'])], i) for i in t_estims_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [4]:
def get_X_from(t_data, t_extract_data):
    t_cleaned_data = []
    clinic_names = []
    ids = []
    for i in t_data:
        if i['doctors']!=[]:
            t_cleaned_data += [t_extract_data(i,[])]
            clinic_names += [unicode(i['name_ru'])] * len(i['doctors'])
            ids += [int(i['id'])] * len(i['doctors'])
    X = sum([i for i in t_cleaned_data],[])
    return X, clinic_names, ids

In [5]:
def get_best_ts(X, y):
    results = pd.DataFrame(columns = ['variance_train', 'variance_test', 'absolute_train', 'absolute_test', 'ts'])
    for i in range(1,7):
        regr = process_with(X, y, return_long=True, ts=i/10.0)
        if ((regr['variance_train'] == 1) and (round(regr['absolute_train'],2) == 0)): continue
        results = results.append(regr,ignore_index=True)
        results['score'] = results.apply(lambda row: abs(((row['absolute_train'] + row['absolute_test'])/2.0)/
                                 ((1 - row['variance_train'])/2.0)*
                                 ((1 - row['variance_test']/2.0)))*
                                 abs((row['absolute_train'] - row['absolute_test'])), axis=1)

    if len(results): return results['ts'][results['score'].argmin()]
    else: return 0

In [6]:
def process_with(X,y, info=False, short=False, return_short = False, return_long = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print "%.3f" % np.mean(abs(regr.predict(train_X) - train_y))
        print "%.3f" % np.mean(abs(regr.predict(test_X) - test_y))
        print "%.3f" % (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100)
        print "%.3f" % regr.score(train_X, train_y)
        print "%.3f" % regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_long:
        return {"absolute_train": np.mean(abs(regr.predict(train_X) - train_y)),
               "absolute_test": np.mean(abs(regr.predict(test_X) - test_y)),
               "variance_train": regr.score(train_X, train_y),
               "variance_test": regr.score(test_X, test_y),
               ""
               "ts": ts}
    #for i in regr.coef_:
        #print "%.3f" % i
    #print "%.3f" % regr.intercept_
    return regr

In [7]:
def pack(X, gd, ed, gp):
    new_X = []
    for x in X:
        new_X+= [sum([
            [round(sum(np.array(gd)*np.array(x[0:7])),5)],
            [round(sum(np.array(ed)*np.array(x[7:12])),5)],
            [round(sum(np.array(gp)*np.array(x[12:16])),5)],
            x[16:]
        ],[])]
    return new_X

In [8]:
def seq_procent(input_array):
    new_pos=np.array(input_array)
    straight = np.array([i for i in range(max(new_pos), min(new_pos)-1, -1)])
    inversions = abs(straight - new_pos)
    while not np.array_equal(inversions, np.zeros(len(inversions))):
        new_pos=np.delete(new_pos, inversions.argmax())
        new_pos = [10 - sorted(new_pos, reverse=True).index(x) for x in new_pos]
        straight = np.array([i for i in range(max(new_pos), min(new_pos)-1, -1)])
        inversions = abs(straight - new_pos)
    return 100*len(new_pos)/float(len(input_array))

In [ ]:
def get_X_sets(extract_data):
    X_all, y_all = [], []

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[2]['clinics']
    with open('%s/меланома_все.json'%this_path) as data_file: #2
        cancer_data = json.load(data_file)[0]['clinics']
    X_1, y_1 = get_X_y_from(extract_data, cancer_data, estims_data)
    X_all+=X_1
    y_all+=y_1

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[0]['clinics']
    with open('%s/рак_груди_все.json'%this_path) as data_file: #0
        cancer_data = json.load(data_file)[0]['clinics']
    X_2, y_2 = get_X_y_from(extract_data, cancer_data, estims_data)
    X_all+=X_2
    y_all+=y_2

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[4]['clinics']    
    with open('%s/рак_простаты_все.json'%this_path) as data_file: #4
        cancer_data = json.load(data_file)[0]['clinics']
    clinic_ids = [i['id'] for i in estims_data]
    estims_data.pop(clinic_ids.index('0'))
    X_3, y_3 = get_X_y_from(extract_data, cancer_data, estims_data)
    X_all+=X_3
    y_all+=y_3

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[1]['clinics']     
    with open('%s/рак_шейки_матки_все.json'%this_path) as data_file: #1
        cancer_data = json.load(data_file)[0]['clinics']
    X_4, y_4 = get_X_y_from(extract_data, cancer_data, estims_data)
    X_all+=X_4
    y_all+=y_4

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[3]['clinics']         
    with open('%s/рак_щитовидки_все.json'%this_path) as data_file: #3
        cancer_data = json.load(data_file)[0]['clinics']
    X_5, y_5 = get_X_y_from(extract_data, cancer_data, estims_data)
    X_all+=X_5
    y_all+=y_5
    
    return X_all, X_1, X_2, X_3, X_4, X_5, y_all, y_1, y_2, y_3, y_4, y_5

In [ ]:
def get_clinic_names(t_estims_data, t_cancer_data):
    ids = [i['id'] for i in t_estims_data]
    if '0' in ids:
        ids.pop('0')
    clinic_names = []
    for id_ in ids:
        clinic_names += [ [i["name_ru"], len(i['doctors'])] for i in t_cancer_data if i['id'] == id_ and len(i['doctors'])!= 0]
    return clinic_names

In [ ]:
def get_clinic_names_all():
    X_all, y_all = [], []

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[2]['clinics']
    with open('%s/меланома_все.json'%this_path) as data_file: #2
        cancer_data = json.load(data_file)[0]['clinics']
    clinic_names_1 = get_clinic_names(estims_data, cancer_data)

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[0]['clinics']
    with open('%s/рак_груди_все.json'%this_path) as data_file: #0
        cancer_data = json.load(data_file)[0]['clinics']
    clinic_names_2 = get_clinic_names(estims_data, cancer_data)

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[4]['clinics']    
    with open('%s/рак_простаты_все.json'%this_path) as data_file: #4
        cancer_data = json.load(data_file)[0]['clinics']
    clinic_ids = [i['id'] for i in estims_data]
    estims_data.pop(clinic_ids.index('0'))
    clinic_names_3 = get_clinic_names(estims_data, cancer_data)

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[1]['clinics']     
    with open('%s/рак_шейки_матки_все.json'%this_path) as data_file: #1
        cancer_data = json.load(data_file)[0]['clinics']
    clinic_names_4 = get_clinic_names(estims_data, cancer_data)

    with open('./max/estims.json') as data_file: 
        estims_data = json.load(data_file)[3]['clinics']         
    with open('%s/рак_щитовидки_все.json'%this_path) as data_file: #3
        cancer_data = json.load(data_file)[0]['clinics']
    clinic_names_5 = get_clinic_names(estims_data, cancer_data)

    
    return clinic_names_1 + clinic_names_2 + clinic_names_3 + clinic_names_4 + clinic_names_5

In [ ]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('%s/меланома_все.json'%this_path) as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
clinic_names_1 = get_clinic_names(estims_data, cancer_data)

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('%s/рак_груди_все.json'%this_path) as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
clinic_names_2 = get_clinic_names(estims_data, cancer_data)

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('%s/рак_простаты_все.json'%this_path) as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
clinic_names_3 = get_clinic_names(estims_data, cancer_data)

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('%s/рак_шейки_матки_все.json'%this_path) as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
clinic_names_4 = get_clinic_names(estims_data, cancer_data)

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('%s/рак_щитовидки_все.json'%this_path) as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
clinic_names_5 = get_clinic_names(estims_data, cancer_data)


return clinic_names_1 + clinic_names_2 + clinic_names_3 + clinic_names_4 + clinic_names_5

In [23]:
sub_oper = []
sub_diag = []
for file_name in ["меланома", "рак_груди", "рак_простаты", "рак_шейки_матки", "рак_щитовидки"]:
    with open('%s/%s_все.json'%(this_path, file_name)) as data_file: #1
        cancer_data = json.load(data_file)[0]['clinics']
        print file_name
        for clinic in cancer_data:
            if clinic['top']==u'1':
                if clinic['doctors']!=[]:
                    print "\t %s" % clinic['name_ru']
                    for doc in clinic['doctors']:
                        if doc['top']==u'1':
                            if len(sum(doc['procedures'].values(), [])) != 0:
                                print "\t \t %s" % doc['name_ru']                        
                                oper = [i['name_ru'] for i in doc['procedures']['operations']]
                                sub_oper += oper
                                if oper != []:
                                    for o in oper:
                                        print "\t \t \t %s - o" % o
                                diag = [i['name_ru'] for i in doc['procedures']['diagnostics']]
                                sub_diag += diag
                                if oper != []:
                                    for o in diag:
                                        print "\t \t \t %s - d" % o

меланома
	 Медицинский центр им. Сураски (Ихилов)
	 	 Илан Гиль Рон
	 	 	 Хирургия меланомы - o
	 	 	 Консультация онколога - d
	 	 Диана Мацеевски
	 	 	 Радиотерапия - o
	 	 	 ПЭТ-КТ - d
	 	 	 МРТ  - d
	 	 	 КТ - d
	 Клиника Ассута
	 	 Моше Инбар
	 	 	 Химиотерапия - o
	 	 	 Брахитерапия - o
	 	 	 Лучевая терапия  - o
	 	 	 Радиотерапия - o
	 	 	 МРТ  - d
	 	 	 Биопсия  - d
	 	 	 Анализ крови на онкомаркеры - d
	 	 	 Биопсия "сторожевых" лимфоузлов - d
	 	 	 Тонкоигольная аспирационная биопсия опухоли  - d
	 Больница им. Хаима Шиба
рак_груди
	 Медицинский центр им. Сураски (Ихилов)
	 	 Илан Гиль Рон
	 	 	 Хирургия меланомы - o
	 	 	 Консультация онколога - d
	 	 Диана Мацеевски
	 	 	 Радиотерапия - o
	 	 	 ПЭТ-КТ - d
	 	 	 МРТ  - d
	 	 	 КТ - d
	 Клиника Ассута
	 	 Моше Инбар
	 	 	 Химиотерапия - o
	 	 	 Брахитерапия - o
	 	 	 Лучевая терапия  - o
	 	 	 Радиотерапия - o
	 	 	 МРТ  - d
	 	 	 Биопсия  - d
	 	 	 Анализ крови на онкомаркеры - d
	 	 	 Биопсия "сторожевых" лимфоузлов - d
	 

In [ ]:
cancer_data[0]['procedures']

In [22]:
sub_oper = []
sub_diag = []
for file_name in ["меланома", "рак_груди", "рак_простаты", "рак_шейки_матки", "рак_щитовидки"]:
    with open('%s/%s_все.json'%(this_path, file_name)) as data_file: #1
        cancer_data = json.load(data_file)[0]['clinics']
        print file_name
        for clinic in cancer_data:
            if clinic['top']==u'1':
                if clinic['procedures']!=[]:
                    oper = [i['name_ru'] for i in clinic['procedures']['operations'] if i['top']==u'1']
                    sub_oper += oper
                    if oper != []:
                        print "\t %s" % clinic['name_ru']
                        #print len(clinic['doctors'])
                        for o in oper:
                            print "\t \t %s" % o

меланома
	 Медицинский центр им. Сураски (Ихилов)
	 	 Артроскопия коленного сустава 
	 	 Аортокоронарное шунтирование
	 	 Гистерэктомия (удаление тела и шейки матки)
	 	 Установка кардиостимулятора
	 Клиника Ассута
	 	 Артроскопия коленного сустава 
	 	 Гистерэктомия (удаление тела и шейки матки)
	 	 Установка кардиостимулятора
	 Медицинский центр Рамбам
	 	 Артроскопия коленного сустава 
	  Частная клиника Кардиолита
	 	 Установка кардиостимулятора
рак_груди
	 Медицинский центр им. Сураски (Ихилов)
	 	 Артроскопия коленного сустава 
	 	 Аортокоронарное шунтирование
	 	 Гистерэктомия (удаление тела и шейки матки)
	 	 Установка кардиостимулятора
	 Клиника Ассута
	 	 Артроскопия коленного сустава 
	 	 Гистерэктомия (удаление тела и шейки матки)
	 	 Установка кардиостимулятора
	 Медицинский центр Рамбам
	 	 Артроскопия коленного сустава 
	  Частная клиника Кардиолита
	 	 Установка кардиостимулятора
рак_простаты
	 Медицинский центр им. Сураски (Ихилов)
	 	 Артроскопия коленного сустава 
	 

In [181]:
for i in set(sub_oper):
    print i

print len(set(sub_oper))

Анализ крови на онкомаркеры
УЗИ
Консультация гинеколога
ПАП-тест (мазок Паппаниколау)
ПЭТ-КТ
Кольпоскопия
КТ
Биопсия "сторожевых" лимфоузлов
Биопсия 
Консультация онколога
Тонкоигольная аспирационная биопсия опухоли 
МРТ 
12


In [186]:
print len(cancer_data[0]['doctors'][0]['procedures']['operations'])
print len(cancer_data[0]['doctors'][0]['procedures']['diagnostics'])
len(sum(cancer_data[0]['doctors'][0]['procedures'].values(), []))

2
10


12

In [25]:
with open('%s/меланома_все.json'%this_path) as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']

#all = cancer_data[0]['procedures']['operations'] #+ cancer_data[0]['procedures']['diagnostics']
all = sum([i['procedures']['operations'] for i in cancer_data],[]) #+ cancer_data[0]['procedures']['diagnostics']
for i in all:
    if i['top'] == u'1':
        print i['name_ru'], i['top']

Артроскопия коленного сустава  1
Аортокоронарное шунтирование 1
Гистерэктомия (удаление тела и шейки матки) 1
Установка кардиостимулятора 1
Артроскопия коленного сустава  1
Гистерэктомия (удаление тела и шейки матки) 1
Установка кардиостимулятора 1
Артроскопия коленного сустава  1
Аортокоронарное шунтирование 1
Гистерэктомия (удаление тела и шейки матки) 1
Аортокоронарное шунтирование 1
Гистерэктомия (удаление тела и шейки матки) 1
Гистерэктомия (удаление тела и шейки матки) 1
Аортокоронарное шунтирование 1
Установка кардиостимулятора 1
Гистерэктомия (удаление тела и шейки матки) 1
Установка кардиостимулятора 1
Артроскопия коленного сустава  1
Артроскопия коленного сустава  1
Установка кардиостимулятора 1
Гистерэктомия (удаление тела и шейки матки) 1
Установка кардиостимулятора 1
Установка кардиостимулятора 1
Установка кардиостимулятора 1
Установка кардиостимулятора 1
Артроскопия коленного сустава  1
Установка кардиостимулятора 1
Гистерэктомия (удаление тела и шейки матки) 1
Артроскопи

In [31]:
names= set([i['name_ru'] for i in all])

In [32]:
for i in names:
    print i

Гамма-нож
 Конизация шейки матки + биопсия
АКШ (Аортокоронарное шунтирование)
Робот-ассистированная радикальная простатэктомия с Да Винчи
Артропластика
Трахелэктомия
Гипертермическая внутрибрюшинная химиотерапия (HIPEC)
Пересадка печени
Химиотерапия
Операция по восстановлению манжета плеча
Субтотальная тиреоидэктомия
Гормональная терапия
Радиочастотная абляция
Удаление рака толстого кишечника
Нефрэктомия
Дискэктомия
Высокоинтенсивная фокусированная ультразвуковая абляция (HIFU)
Феморально–поплитеальное шунтирование
Роботизированная система Да Винчи
Бариатрия (уменьшение желудка)
Стентирование (1 стент)
Операции при раке молочной железы
Эндопротезирование коленного сустава
Разрушение пучков катетером (электрофизиология)
Лучевая терапия 
Полная замена тазобедренного сустава
Гистерэктомия (удаление тела и шейки матки)
Лампэктомия
Удаление опухоли спинного мозга
Радиойодтерапия
Таргетная терапия
SIRT терапия
Простатэктомия
Реконструкция передней крестообразной связки
Брахитерапия
Удаление 

In [41]:
for i in cancer_data[0]['procedures']['operations']:
    print i['name_ru'], i['top']

Замена сердечного клапана 0
Мастэктомия 0
Удаление опухоли простаты (обычное) 0
Химиотерапия 0
Полная замена коленного сустава 0
Полная замена тазобедренного сустава 0
Дискэктомия 0
Артроскопия коленного сустава  1
Удаление опухоли головного мозга 0
Брахитерапия 0
Восстановительная хирургия груди 0
Гамма-нож 0
Кибер-нож 0
Высокоинтенсивная фокусированная ультразвуковая абляция (HIFU) 0
Простатэктомия 0
Протонно-лучевая терапия 0
Радиочастотная абляция 0
Удаление опухоли спинного мозга 0
Хирургия меланомы 0
Нефрэктомия 0
Гастрэктомия 0
Цистэктомия 0
Роботизированная система Да Винчи 0
Аортокоронарное шунтирование 1
Лучевая терапия  0
Глубокая стимуляция мозга (DBS) 0
Маммопластика 0
Биологическая терапия 0
ТУР (трансуретральная резекция) простаты 0
Радиойодтерапия 0
Операции при раке молочной железы 0
Трахелэктомия 0
Гистерэктомия (удаление тела и шейки матки) 1
Гормональная терапия 0
Радикальная простатэктомия 0
Иммунотерапия 0
Таргетная терапия 0
Изолированная перфузия конечности 0
Ла